# TP Transformers

author :<br />
Gousset Johan<br />
Hamon Maxime<br />
Ouachene Naomi<br />
Pénisson Clément<br />

# Introduction to transformers
### Build your own translator from French to Portuguese

source : tensorflow

This TP will guide you through the Transformers's get started.<br>
You will have to complete some blanks marked with "TODO".<br>

Of course, you will need way more training time to achieve decent results. This first part is here to help you understand what's inside a transformer.<br>

The second part will show you how well pretrained models on huge datasets with a lot of training time performs.<br>

In [ ]:
pip install tfds-nightly

In [ ]:
pip install matplotlib==3.2.2

In [ ]:
pip install transformers==3.5.0

In [ ]:
import tensorflow_datasets as tfds
import tensorflow as tf

import time
import numpy as np
import matplotlib.pyplot as plt

Retrieve datas from https://www.tensorflow.org/datasets/catalog/ted_hrlr_translate

In [ ]:
#langs = "fr_to_pt"
langs = "pt_to_en"

examples, metadata = tfds.load("ted_hrlr_translate/" + langs, with_info=True, as_supervised=True)
train_examples, val_examples = examples['train'], examples['validation']

Shuffling and writing examples to /root/tensorflow_datasets/ted_hrlr_translate/pt_to_en/1.0.0.incomplete7WQPS2/ted_hrlr_translate-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/ted_hrlr_translate/pt_to_en/1.0.0.incomplete7WQPS2/ted_hrlr_translate-validation.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/ted_hrlr_translate/pt_to_en/1.0.0.incomplete7WQPS2/ted_hrlr_translate-test.tfrecord


Dataset ted_hrlr_translate downloaded and prepared to /root/tensorflow_datasets/ted_hrlr_translate/pt_to_en/1.0.0. Subsequent calls will reuse this data.


In [ ]:
tokenizer_lang_src = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus((lang1.numpy() for lang1, lang2 in train_examples), target_vocab_size=2**13)
tokenizer_lang_dest = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus((lang2.numpy() for lang1, lang2 in train_examples), target_vocab_size=2**13)

In [ ]:
BUFFER_SIZE = 20000
BATCH_SIZE = 64
MAX_LENGTH = 40

In [ ]:
# To keep the training phase fast enough, delete the sentences that are too long
def remove_long_sentences(s1, s2):
  return tf.logical_and(tf.size(s1) <= MAX_LENGTH, tf.size(s2) <= MAX_LENGTH)

def encode(lang1, lang2):
  lang1 = [tokenizer_lang_src.vocab_size] + tokenizer_lang_src.encode(lang1.numpy()) + [tokenizer_lang_src.vocab_size+1]
  lang2 = [tokenizer_lang_dest.vocab_size] + tokenizer_lang_dest.encode(lang2.numpy()) + [tokenizer_lang_dest.vocab_size+1]
  return lang1, lang2

def tf_encode(src, dest):  
  result_src, result_dest = tf.py_function(encode, [src, dest], [tf.int64, tf.int64])
  result_src.set_shape([None])
  result_dest.set_shape([None])
  return result_src, result_dest

train_dataset = train_examples.map(tf_encode).filter(remove_long_sentences).cache()
train_dataset = train_dataset.shuffle(BUFFER_SIZE).padded_batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)

val_dataset = val_examples.map(tf_encode).padded_batch(BATCH_SIZE)

The encoders calculate the outputs for each token without taking into account their position in the sentence, so positional encoding must be added to overcome this problem. It's up to you to find out how to calculate the latter.

Reminder, the formula is : <br>
$PE_{(pos, 2i)} = sin(\frac{pos}{10000^{\frac{2i}{d_{model}}}})$ <br><br>
$PE_{(pos, 2i+1)} = cos(\frac{pos}{10000^{\frac{2i}{d_{model}}}})$ <br>

d : dimension

In [ ]:
def positional_encoding(position, dim_model):
    #TODO
    return tf.cast(angles[np.newaxis, ...], dtype=tf.float32)

In [ ]:
# Test 1
pos_encoding = positional_encoding(3, 10)
result = np.array([[[0, 1, 0, 1, 0, 1, 0, 1, 0, 1],
                    [0.841470, 0.540302, 0.157826, 0.987466, 0.025116, 
                     0.999684, 0.00398106, 0.999992, 0.00063095, 0.999999], 
                    [0.909297, -0.416146, 0.311697, 0.950181, 0.0502166, 
                     0.998738, 0.007962, 0.999968, 0.0012619, 0.999999]]])
assert np.allclose(result, pos_encoding)

# Test 2
pos_encoding = positional_encoding(7, 12)
result = np.array([[[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1],
                    [0.84147096, 0.5403023, 0.21378067, 0.9768817, 0.046399225, 0.998923, 
                     0.009999833, 0.99995, 0.002154433, 0.9999977, 0.00046415886, 0.9999999],
                    [0.9092974, -0.41614684, 0.41767684, 0.9085957, 0.0926985, 0.9956942, 
                     0.019998666, 0.9998, 0.004308856, 0.9999907, 0.00092831766, 0.9999996],
                    [0.14112, -0.9899925, 0.602261, 0.7982992, 0.1387981, 0.9903207, 
                     0.029995501, 0.99955004, 0.006463259, 0.99997914, 0.0013924761, 0.99999905],
                    [-0.7568025, -0.6536436, 0.7589987, 0.6510921, 0.18459873, 0.98281395, 
                     0.039989334, 0.9992001, 0.008617632, 0.99996287, 0.0018566345, 0.9999983],
                    [-0.9589243, 0.2836622, 0.88064283, 0.47378072, 0.23000172, 0.97319025, 
                     0.04997917, 0.99875027, 0.010771965, 0.999942, 0.0023207923, 0.9999973],
                    [-0.2794155, 0.96017027, 0.961569, 0.27456328, 0.27490926, 0.9614702, 
                     0.059964005, 0.99820054, 0.012926248, 0.99991643, 0.0027849497, 0.9999961]]])
assert np.allclose(result, pos_encoding)

print("Your positional encoding is looking great, you can continue !")

Your positional encoding is looking great, you can continue !


In [ ]:
def create_padding_mask(seq):
  seq = tf.cast(tf.math.equal(seq, 0), tf.float32)
  return seq[:, tf.newaxis, tf.newaxis, :]

def create_look_ahead_mask(size):
  return 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)

## Scaled dot product attention

Now, try to compute the attention weights : Attention(Q,K,V)<br>
Q is the Query matrix<br>
K is the Keys matrix<br>
V is the Values matrix<br>
The returns value should be the attention_output and the attention_weights<br>

Reminder, The formula is :<br>
$Attention(Q, K, V) = softmax(\frac{QK^T}{\sqrt{d_k}}) * V$

Advices :<br>
The attention weights don't need the Values matrix<br>
To apply the mask, simply add $(mask * -1e9)$ before applying the function you need to use

In [ ]:
def compute_attention_weights(Q, K, V, mask=None):
  # TODO
  return attention_output, attention_weights

Set of tests to ensure your attention result and weights are corrects

In [ ]:
np.set_printoptions(suppress=True)

k_test = tf.constant([[10,0,0],
                      [0,10,0],
                      [0,0,10],
                      [0,0,10]], dtype=tf.float32)  # (4, 3)

v_test = tf.constant([[   1,0],
                      [  10,0],
                      [ 100,5],
                      [1000,6]], dtype=tf.float32)  # (4, 2)

# This `query` aligns with the second `key`,
# so the second `value` is returned.
q_test = tf.constant([[0, 10, 0]], dtype=tf.float32)
output_test, weights_test = compute_attention_weights(q_test, k_test, v_test)
print("your result     : ", weights_test.numpy(), ", ", output_test.numpy(), sep="")
print("expected result : [[0. 1. 0. 0.]], [[10.  0.]]\n")
assert np.allclose(weights_test.numpy(), [[0, 1, 0, 0]]) and np.allclose(output_test.numpy(), [[10, 0]])


# This query aligns with a repeated key (third and fourth), 
# so all associated values get averaged.
q_test = tf.constant([[0, 0, 10]], dtype=tf.float32)
output_test, weights_test = compute_attention_weights(q_test, k_test, v_test)
print("your result     : ", weights_test.numpy(), ", ", output_test.numpy(), sep="")
print("expected result : [[0.  0.  0.5 0.5]], [[550.    5.5]]\n")
assert np.allclose(weights_test.numpy(), [[0, 0, 0.5, 0.5]]) and np.allclose(output_test.numpy(), [[550, 5.5]])

# This query aligns equally with the first and second key, 
# so their values get averaged.
q_test = tf.constant([[10, 10, 0]], dtype=tf.float32)
output_test, weights_test = compute_attention_weights(q_test, k_test, v_test)
print("your result     : ", weights_test.numpy(), ", ", output_test.numpy(), sep="")
print("expected result : [[0.5 0.5 0.  0. ]], [[5.5 0. ]]\n")
assert np.allclose(weights_test.numpy(), [[0.5, 0.5, 0, 0]]) and np.allclose(output_test.numpy(), [[5.5, 0]])

q_test = tf.constant([[0, 0, 10], [0, 10, 0], [10, 10, 0]], dtype=tf.float32)
output_test, weights_test = compute_attention_weights(q_test, k_test, v_test)
print("your result :\n", weights_test.numpy(), "\n", output_test.numpy(), sep="")
print("expected result :\n[[0.  0.  0.5 0.5]\n[0.  1.  0.  0. ]\n[0.5 0.5 0.  0. ]]\n[[550.    5.5]\n[ 10.    0. ]\n[  5.5   0. ]]\n")
assert np.allclose(weights_test.numpy(), [[0, 0, 0.5, 0.5],[0, 1, 0, 0],[0.5, 0.5, 0, 0]]) and np.allclose(output_test.numpy(), [[550, 5.5],[10, 0],[5.5, 0]])

your result     : [[0. 1. 0. 0.]], [[10.  0.]]
expected result : [[0. 1. 0. 0.]], [[10.  0.]]

your result     : [[0.  0.  0.5 0.5]], [[550.    5.5]]
expected result : [[0.  0.  0.5 0.5]], [[550.    5.5]]

your result     : [[0.5 0.5 0.  0. ]], [[5.5 0. ]]
expected result : [[0.5 0.5 0.  0. ]], [[5.5 0. ]]

your result :
[[0.  0.  0.5 0.5]
 [0.  1.  0.  0. ]
 [0.5 0.5 0.  0. ]]
[[550.    5.5]
 [ 10.    0. ]
 [  5.5   0. ]]
expected result :
[[0.  0.  0.5 0.5]
[0.  1.  0.  0. ]
[0.5 0.5 0.  0. ]]
[[550.    5.5]
[ 10.    0. ]
[  5.5   0. ]]



## Multi-head attention



In [ ]:
class MultiHeadAttention(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads):
    super(MultiHeadAttention, self).__init__()
    self.num_heads = num_heads
    self.d_model = d_model
    assert d_model % self.num_heads == 0
    self.depth = d_model // self.num_heads
    self.wq = tf.keras.layers.Dense(d_model)
    self.wk = tf.keras.layers.Dense(d_model)
    self.wv = tf.keras.layers.Dense(d_model)
    self.dense = tf.keras.layers.Dense(d_model)

  """
  Split the last dimension into (num_heads, depth).
  Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
  """
  def split_heads(self, x, batch_size):
    x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(x, perm=[0, 2, 1, 3])

  def call(self, v, k, q, mask):
    batch_size = tf.shape(q)[0]

    q = self.wq(q)  
    k = self.wk(k)
    v = self.wv(v)
    q = self.split_heads(q, batch_size)
    k = self.split_heads(k, batch_size)
    v = self.split_heads(v, batch_size)

    scaled_attention, attention_weights = compute_attention_weights(q, k, v, mask)
    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])
    concat_attention = tf.reshape(scaled_attention, (batch_size, -1, self.d_model))
    output = self.dense(concat_attention)
    return output, attention_weights

![image](https://miro.medium.com/max/2880/1*BHzGVskWGS_3jEcYYi6miQ.png)
This schema might be helpful 😉








## Encoder


In [ ]:
class EncoderLayer(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads, dim_feed_forward, rate=0.1):
    super(EncoderLayer, self).__init__()
    # TODO
    # Use 1e-6 epsilon for normalisation

    self.dropout1 = tf.keras.layers.Dropout(rate)
    self.dropout2 = tf.keras.layers.Dropout(rate)

  def call(self, x, training, mask):
    attention_output, _ = self.multi_head_attention(x, x, x, mask)
    attention_output = self.dropout1(attention_output, training=training)
    first_output = self.layernorm1(x + attention_output)

    feed_forward_output = self.feed_forward_network(first_output)
    feed_forward_output = self.dropout2(feed_forward_output, training=training)
    output = self.layernorm2(first_output + feed_forward_output)
    return output

class Encoder(tf.keras.layers.Layer):
  def __init__(self, num_layers, d_model, num_heads, dim_feed_forward, input_vocab_size,
               maximum_position_encoding, rate=0.1):
    super(Encoder, self).__init__()
    self.d_model = d_model
    self.num_layers = num_layers
    # TODO
    # Define parts of the encoder
    
    self.dropout = tf.keras.layers.Dropout(rate)

  def call(self, x, training, mask):
    seq_len = tf.shape(x)[1]

    # adding embedding and position encoding.
    x = self.embedding(x)
    x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
    x += self.pos_encoding[:, :seq_len, :]
    x = self.dropout(x, training=training)

    for i in range(self.num_layers):
      x = self.enc_layers[i](x, training, mask)

    return x  # (batch_size, input_seq_len, d_model)

## Decoder

In [ ]:
class DecoderLayer(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads, dim_feed_forward, rate=0.1):
    super(DecoderLayer, self).__init__()
    # TODO
    # Use 1e-6 epsilon for normalisation

    self.dropout1 = tf.keras.layers.Dropout(rate)
    self.dropout2 = tf.keras.layers.Dropout(rate)
    self.dropout3 = tf.keras.layers.Dropout(rate)
  
  def call(self, x, enc_output, training, look_ahead_mask, padding_mask):
    # enc_output.shape == (batch_size, input_seq_len, d_model)
    attention1, attention_weights_block1 = self.multi_head_attention1(x, x, x, look_ahead_mask)
    attention1 = self.dropout1(attention1, training=training)
    output1 = self.layernorm1(attention1 + x)

    attention2, attention_weights_block2 = self.multi_head_attention1(enc_output, enc_output, output1, padding_mask)
    attention2 = self.dropout2(attention2, training=training)
    output2 = self.layernorm2(attention2 + output1)

    feed_forward_output = self.feed_forward_network(output2)
    feed_forward_output = self.dropout3(feed_forward_output, training=training)
    output = self.layernorm3(feed_forward_output + output2)
    return output, attention_weights_block1, attention_weights_block2


class Decoder(tf.keras.layers.Layer):
  def __init__(self, num_layers, d_model, num_heads, dim_feed_forward, target_vocab_size,
               maximum_position_encoding, rate=0.1):
    super(Decoder, self).__init__()

    self.d_model = d_model
    self.num_layers = num_layers
    
    # TODO
    # Define parts of the decoder

    
    self.dropout = tf.keras.layers.Dropout(rate)

  def call(self, x, enc_output, training, look_ahead_mask, padding_mask):
    seq_len = tf.shape(x)[1]
    attention_weights = {}

    x = self.embedding(x)  # (batch_size, target_seq_len, d_model)
    x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
    x += self.pos_encoding[:, :seq_len, :]

    x = self.dropout(x, training=training)

    for i in range(self.num_layers):
      x, block1, block2 = self.dec_layers[i](x, enc_output, training, look_ahead_mask, padding_mask)

      attention_weights['decoder_layer{}_block1'.format(i+1)] = block1
      attention_weights['decoder_layer{}_block2'.format(i+1)] = block2

    # x.shape == (batch_size, target_seq_len, d_model)
    return x, attention_weights

## Transformer

Now that you have define the encoder and decoder, build your own transformer.

Advices: <br>
Don't forget the last layer. The number of neurons is equals to target_vocab_size.

In [ ]:
class Transformer(tf.keras.Model):
  def __init__(self, num_layers, d_model, num_heads, dim_feed_forward, input_vocab_size, 
               target_vocab_size, pe_input, pe_target, rate=0.1):
    super(Transformer, self).__init__()
    # TODO
    #Define parts of the transformers
    

  def call(self, inp, tar, training, enc_padding_mask, 
           look_ahead_mask, dec_padding_mask):
    # TODO
    # Compute the results

    
    return output, attention_weights

In [ ]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
  def __init__(self, d_model, warmup_steps=4000):
    super(CustomSchedule, self).__init__()
    self.d_model = d_model
    self.d_model = tf.cast(self.d_model, tf.float32)
    self.warmup_steps = warmup_steps

  def __call__(self, step):
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps ** -1.5)
    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [ ]:
num_layers = 4
d_model = 128
dim_feed_forward = 512
num_heads = 8
input_vocab_size = tokenizer_lang_src.vocab_size + 2
target_vocab_size = tokenizer_lang_dest.vocab_size + 2
dropout_rate = 0.1
learning_rate = CustomSchedule(d_model)
optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

In [ ]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_sum(loss_)/tf.reduce_sum(mask)

def accuracy_function(real, pred):
  accuracies = tf.equal(real, tf.argmax(pred, axis=2))

  mask = tf.math.logical_not(tf.math.equal(real, 0))
  accuracies = tf.math.logical_and(mask, accuracies)

  accuracies = tf.cast(accuracies, dtype=tf.float32)
  mask = tf.cast(mask, dtype=tf.float32)
  return tf.reduce_sum(accuracies)/tf.reduce_sum(mask)

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.Mean(name='train_accuracy')

In [ ]:
transformer = Transformer(num_layers, d_model, num_heads, dim_feed_forward,
                          input_vocab_size, target_vocab_size, 
                          pe_input=input_vocab_size, 
                          pe_target=target_vocab_size,
                          rate=dropout_rate)

In [ ]:
def create_masks(inp, tar):
  enc_padding_mask = create_padding_mask(inp)

  # Used in the 2nd attention block in the decoder.
  # This padding mask is used to mask the encoder outputs.
  dec_padding_mask = create_padding_mask(inp)

  # Used in the 1st attention block in the decoder.
  # It is used to pad and mask future tokens in the input received by the decoder.
  look_ahead_mask = create_look_ahead_mask(tf.shape(tar)[1])
  dec_target_padding_mask = create_padding_mask(tar)
  combined_mask = tf.maximum(dec_target_padding_mask, look_ahead_mask)

  return enc_padding_mask, combined_mask, dec_padding_mask

In [ ]:
# Use predefined train steps
train_step_signature = [
    tf.TensorSpec(shape=(None, None), dtype=tf.int64),
    tf.TensorSpec(shape=(None, None), dtype=tf.int64),
]

@tf.function(input_signature=train_step_signature)
def train_step(inp, tar):
  tar_inp = tar[:, :-1]
  tar_real = tar[:, 1:]

  enc_padding_mask, combined_mask, dec_padding_mask = create_masks(inp, tar_inp)

  with tf.GradientTape() as tape:
    predictions, _ = transformer(inp, tar_inp, 
                                 True, 
                                 enc_padding_mask, 
                                 combined_mask, 
                                 dec_padding_mask)
    loss = loss_function(tar_real, predictions)

  gradients = tape.gradient(loss, transformer.trainable_variables)    
  optimizer.apply_gradients(zip(gradients, transformer.trainable_variables))

  train_loss(loss)
  train_accuracy(accuracy_function(tar_real, predictions))

In [ ]:
# Manually train the transformer
nb_epochs = 20

for e in range(nb_epochs):
  train_loss.reset_states()
  train_accuracy.reset_states()

  print("Epoch", e+1, "...")
  for (batch, (src_lang, dest_lang)) in enumerate(train_dataset):
    if (batch%50==0):
      print("\tBatch", batch, "...\r")
    train_step(src_lang, dest_lang)
  
  print("Epoch : {}, Loss : {}, Accuracy : {}".format(e+1, train_loss.result(), train_accuracy.result()))

In [ ]:
def evaluate(text):
  # Surround input text with src_lang start and end tokens
  text = [tokenizer_lang_src.vocab_size] + tokenizer_lang_src.encode(text) + [tokenizer_lang_src.vocab_size + 1]
  encoder_input = tf.expand_dims(text, 0)

  decoder_input = [tokenizer_lang_dest.vocab_size] # dest_lang start token
  output = tf.expand_dims(decoder_input, 0)

  for i in range(MAX_LENGTH):
    enc_padding_mask, combined_mask, dec_padding_mask = create_masks(encoder_input, output)

    #predictions.shape == (batch_size, seq_len, vocab_size)
    predictions, attention_weights = transformer(encoder_input, 
                                                 output,
                                                 False,
                                                 enc_padding_mask,
                                                 combined_mask,
                                                 dec_padding_mask)

    # Select the last word from the seq_len dimension
    predictions = predictions[: ,-1:, :]  # (batch_size, 1, vocab_size)
    predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

    # Return the result if the predicted_id is equal to the end token
    if predicted_id == tokenizer_lang_dest.vocab_size+1:
      return tf.squeeze(output, axis=0), attention_weights

    # Concatentate the predicted_id to the output which is given to the decoder
    # as its input.
    output = tf.concat([output, predicted_id], axis=-1)

  return tf.squeeze(output, axis=0), attention_weights

In [ ]:
def translate(sentence):
  result, _ = evaluate(sentence)
  return tokenizer_lang_dest.decode([i for i in result if i < tokenizer_lang_dest.vocab_size])  

In [ ]:
# FR -> PT
#print(translate("C'est un problème que l'on doit résoudre."))
#print("Correct translation : este é um problema que temos que resolver.")

# PT -> EN
print(translate("Olá"))
print("Correct translation : Hello")

i strive there .
Correct translation : Hello


# Examples of BERT and GPT-2 below
source : huggingface documentation

In [ ]:
import torch

## BERT

In [ ]:
from transformers import BertTokenizer, BertForMaskedLM

# Load pre-trained model in evaluation mode
model = BertForMaskedLM.from_pretrained('bert-base-uncased')
model.eval()
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

mask = "[MASK]"

def mask_words(text, *indices):
    text = "[CLS]" + text + "[SEP]"
    tokenized_text = tokenizer.tokenize(text)
    for i in indices:
        tokenized_text[i+1] = mask
    return tokenized_text

def predict_masked_words(text):
    indexed_tokens = tokenizer.convert_tokens_to_ids(text)
    segments_ids = [0] * len(indexed_tokens)
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])
    # Forecast all tags
    with torch.no_grad():
        outputs = model(tokens_tensor, token_type_ids=segments_tensors)
        predictions = outputs[0]
    
    predicted_index = [torch.argmax(predictions[0, i]).item() for i,w in enumerate(text) if (w==mask)]
    return tokenizer.convert_ids_to_tokens(predicted_index)

text = "Donald Trump is the USA president."
masked_text = mask_words(text, 1)
print("\nOriginal sentence :", text)
print("Masked text :", masked_text)
print("Predicted text :", predict_masked_words(masked_text))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).




Original sentence : Donald Trump is the USA president.
Masked text : ['[CLS]', 'donald', '[MASK]', 'is', 'the', 'usa', 'president', '.', '[SEP]']
Predicted text : ['trump']


## GPT-2

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Load pre training model (weight)
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

#Load a GPT-2 model in evaluation mode
model = GPT2LMHeadModel.from_pretrained('gpt2')
model.eval()

def predict_next_word(text):
    indexed_tokens = tokenizer.encode(text)
    tokens_tensor = torch.tensor([indexed_tokens])
    # Forecast all tags
    with torch.no_grad():
        outputs = model(tokens_tensor)
        predictions = outputs[0]
    # The next subword to get the prediction
    predicted_index = torch.argmax(predictions[0, -1, :]).item()
    return tokenizer.decode(indexed_tokens + [predicted_index])

text = "Barack Obama is a"
print("original text :", text)
print("predicted text :", predict_next_word(text))


original text : Barack Obama is a
predicted text : Barack Obama is a man


Example of GPT-2's text prediction from an original text using the past

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained('gpt2')

def print_prediction_using_past(text, nb_iter=50, show=True):
    generated = tokenizer.encode(text)
    context = torch.tensor([generated])
    past = None
    for i in range(nb_iter):
        output, past = model(context, past_key_values=past)
        print(output)
        token = torch.argmax(output[..., -1, :])
        generated += [token.tolist()]
        if (show):
            print(tokenizer.decode(generated))
        context = token.unsqueeze(0)
    return tokenizer.decode(generated)

In [ ]:
print_prediction_using_past("The Manhattan bridge", nb_iter=24)

tensor([[[ -36.2874,  -35.0114,  -38.0793,  ...,  -40.5163,  -41.3759,
           -34.9193],
         [ -67.1545,  -64.9323,  -70.2428,  ...,  -69.9055,  -72.2318,
           -66.0469],
         [ -97.1713,  -96.1762, -101.5702,  ..., -106.6973, -102.6482,
           -98.4244]]], grad_fn=<UnsafeViewBackward>)
The Manhattan bridge is
tensor([[-121.7683, -121.2265, -124.9890,  ..., -130.7813, -123.9292,
         -123.8798]], grad_fn=<MmBackward>)
The Manhattan bridge is a
tensor([[-116.9089, -115.3690, -119.5680,  ..., -125.0401, -119.5787,
         -117.2604]], grad_fn=<MmBackward>)
The Manhattan bridge is a major
tensor([[-118.5036, -115.3115, -123.2127,  ..., -122.8565, -122.8690,
         -119.4754]], grad_fn=<MmBackward>)
The Manhattan bridge is a major artery
tensor([[ -91.1930,  -91.4131,  -97.5173,  ..., -102.5804, -100.4954,
          -93.9918]], grad_fn=<MmBackward>)
The Manhattan bridge is a major artery for
tensor([[-111.4451, -109.5761, -114.5429,  ..., -111.8115, -115.1869,

"The Manhattan bridge is a major artery for the city's subway system, and the bridge is one of the busiest in the country.\n"

In [ ]:
print(print_prediction_using_past("Donald Trump", show=False))

tensor([[[ -33.7367,  -33.0392,  -36.5262,  ...,  -41.7426,  -40.8930,
           -33.9045],
         [ -92.9669,  -90.5406,  -97.7745,  ..., -104.0740,  -98.0077,
           -94.0579]]], grad_fn=<UnsafeViewBackward>)
tensor([[-92.7358, -90.0307, -96.1514,  ..., -96.7022, -96.3673, -91.3721]],
       grad_fn=<MmBackward>)
tensor([[-109.2512, -108.5681, -116.5603,  ..., -118.6861, -115.1757,
         -110.4129]], grad_fn=<MmBackward>)
tensor([[-118.3985, -116.2919, -124.2028,  ..., -122.6029, -118.6487,
         -119.0565]], grad_fn=<MmBackward>)
tensor([[-114.7300, -111.5962, -117.9271,  ..., -119.8242, -116.3627,
         -115.5573]], grad_fn=<MmBackward>)
tensor([[-113.0092, -109.2765, -113.8260,  ..., -118.3055, -116.2692,
         -110.8608]], grad_fn=<MmBackward>)
tensor([[-114.5715, -110.4742, -117.1084,  ..., -116.1955, -120.6147,
         -113.4211]], grad_fn=<MmBackward>)
tensor([[-42.1101, -39.4537, -45.8929,  ..., -45.7747, -49.0152, -41.2206]],
       grad_fn=<MmBackward>)
